In [401]:
# Подгрузим основные библиотеки

In [402]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
#from src.metrics import precision_at_k, recall_at_k
#from src.utils import prefilter_items
#from src.recommenders import MainRecommender

In [405]:
# отберем для тестового датасета 3 недели

In [406]:
data = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 3/test/retail_train.csv')
item_features = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 3/test/product.csv')
user_features = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 3/test/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
#val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks)]
data_test = data[data['week_no'] >= data['week_no'].max() - (val_lvl_2_size_weeks)] 

#data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
#data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]


In [407]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [408]:
data_train.shape

(2278490, 12)

In [409]:
data_test.shape

(118314, 12)

In [410]:
# recommenders

import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                        N=N,
                                        filter_already_liked_items=False,
                                        filter_items=[self.itemid_to_id[999999]],
                                        recalculate_user=True)]

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [411]:
# metrics

import numpy as np


def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    

    bought_list = bought_list  # Тут нет [:k] !!
    
    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    # your_code
    # Лучше считать через скалярное произведение, а не цикл

    return precision


def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)

    return recall


def recall_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)

    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    # your_code

    return recall

In [412]:
# utils

import pandas as pd
import numpy as np


def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.2].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.02].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    # Уберем товары, которые не продавались за последние 12 месяцев

    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features.\
                                        groupby('department')['item_id'].nunique().\
                                        sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]


    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 50]

    # Возбмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    
    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999
    
    # ...

    return data


def postfilter_items(user_id, recommednations):
    pass

In [413]:
data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [414]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [415]:
# отберем топ-5000 популярных товаров 

In [416]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [417]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [418]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [422]:
# проверим модель als

In [423]:
%%time

model = AlternatingLeastSquares(factors=44, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
         show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                       user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                       N=5, # кол-во рекомендаций 
                       filter_already_liked_items=False, 
                       filter_items=None, 
                       recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: total: 9 s
Wall time: 1.46 s


In [424]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [425]:
%%time
    
result_test['als'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result_test.head(5)

CPU times: total: 46.2 s
Wall time: 11.6 s


,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005186, 9526410, 965766, 995242, 5569374]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 951590, 883404, 1133018, 822407]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 1024306, 878996, 965267, 860776]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1058997, 6534178, 1126899, 883404]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1070820, 883404, 862349, 1098066, 826249]"


In [426]:
als = result_test.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
als

0.1871694417238002

In [429]:
# добавим tfidf-взвешивание и пересчитаем модель

In [430]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [431]:
%%time

model = AlternatingLeastSquares(factors=44, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: total: 8.78 s
Wall time: 1.19 s


In [432]:
result_test['als_tfidf'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result_test.head(5)

,user_id,actual,als,als_tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005186, 9526410, 965766, 995242, 5569374]","[1082185, 995242, 1127831, 1051211, 981760]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 951590, 883404, 1133018, 822407]","[951590, 1029743, 1082185, 1106523, 1092026]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 1024306, 878996, 965267, 860776]","[930118, 878996, 1024306, 965267, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1058997, 6534178, 1126899, 883404]","[1082185, 6534178, 1029743, 995242, 1127831]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1070820, 883404, 862349, 1098066, 826249]","[840361, 844179, 995242, 1082185, 1068719]"


In [433]:
als_tfidf = result_test.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()
als_tfidf

0.20362389813907933

In [435]:
# добавим bm25-взвешивание и снова пересчитаем модель

In [436]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [437]:
%%time

model = AlternatingLeastSquares(factors=44, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: total: 10.2 s
Wall time: 1.51 s


In [438]:
result_test['als_bm25'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result_test.head(5)

,user_id,actual,als,als_tfidf,als_bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005186, 9526410, 965766, 995242, 5569374]","[1082185, 995242, 1127831, 1051211, 981760]","[1082185, 995242, 5569374, 8090541, 934369]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 951590, 883404, 1133018, 822407]","[951590, 1029743, 1082185, 1106523, 1092026]","[1029743, 951590, 1092026, 883404, 8090521]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 1024306, 878996, 965267, 860776]","[930118, 878996, 1024306, 965267, 1127831]","[1082185, 1024306, 965267, 857006, 1007195]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1058997, 6534178, 1126899, 883404]","[1082185, 6534178, 1029743, 995242, 1127831]","[1082185, 6534178, 1029743, 995242, 1058997]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1070820, 883404, 862349, 1098066, 826249]","[840361, 844179, 995242, 1082185, 1068719]","[1082185, 1053690, 840361, 995242, 1029743]"


In [439]:
als_bm25 = result_test.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
als_bm25

0.21655239960822725

In [440]:
# теперь попробуем модель own recommender по bm25 взвешивании

In [441]:
%%time

own = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

own.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[10],
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5,
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: total: 1.16 s
Wall time: 323 ms


In [442]:
result_test['own_bm25'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=own, N=5))
result_test.head(5)

,user_id,actual,als,als_tfidf,als_bm25,own_bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1005186, 9526410, 965766, 995242, 5569374]","[1082185, 995242, 1127831, 1051211, 981760]","[1082185, 995242, 5569374, 8090541, 934369]","[480014, 856942, 5577022, 9297615, 1074612]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 951590, 883404, 1133018, 822407]","[951590, 1029743, 1082185, 1106523, 1092026]","[1029743, 951590, 1092026, 883404, 8090521]","[480014, 998206, 1092937, 1092026, 921345]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1007195, 1024306, 878996, 965267, 860776]","[930118, 878996, 1024306, 965267, 1127831]","[1082185, 1024306, 965267, 857006, 1007195]","[480014, 1119051, 13003092, 9911484, 863447]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1058997, 6534178, 1126899, 883404]","[1082185, 6534178, 1029743, 995242, 1127831]","[1082185, 6534178, 1029743, 995242, 1058997]","[480014, 9338009, 6944571, 1122358, 1075524]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1070820, 883404, 862349, 1098066, 826249]","[840361, 844179, 995242, 1082185, 1068719]","[1082185, 1053690, 840361, 995242, 1029743]","[480014, 1116578, 969932, 1105433, 981660]"


In [443]:
own_bm25 = result_test.apply(lambda row: precision_at_k(row['own_bm25'], row['actual']), axis=1).mean()
own_bm25

0.2510610512569377

In [444]:
from prettytable import PrettyTable
x = PrettyTable()

In [445]:
x.field_names = ["als", "als_tfidf", "als_bm25", "own_bm25"]
x.add_row([als, als_tfidf, als_bm25, own_bm25])
x

als,als_tfidf,als_bm25,own_bm25
0.1871694417238002,0.20362389813907933,0.21655239960822725,0.2510610512569377


In [449]:
# модель похожих товаров из-под капота als с bm25 взвешиванием дает наибольший результат precision_at_k на тесте

In [447]:
result = result_test.drop(['actual', 'als', 'als_tfidf', 'als_bm25'], axis=1)

In [448]:
result.to_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 8/recommendations.csv', header=True, index=False, encoding='utf-8')